In [1]:
import os
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from statsmodels.tsa.seasonal import seasonal_decompose 
from statsmodels.tsa.stattools import adfuller

import pmdarima as pm
import statsmodels.api as sm

from src.visualization.metrics import PredictionEvaluator, GlobalResults
from src.config import Config

## Load the data

In [2]:
config = Config()
train_df = pd.read_csv(os.path.join(config.output_cleaned_basic, f'train.csv'))
test_df = pd.read_csv(os.path.join(config.output_cleaned_basic, f'test.csv'))


In [3]:
train_df

,country_order,year,country,co2_actual,co2_predicted
0,0,1852,Stany Zjednoczone,941.144,886.430
1,0,1853,Stany Zjednoczone,955.221,941.144
2,0,1854,Stany Zjednoczone,996.135,955.221
3,0,1855,Stany Zjednoczone,1012.554,996.135
4,0,1856,Stany Zjednoczone,1044.748,1012.554
...,...,...,...,...,...
3193,44,1996,Szwecja,55.760,53.716
3194,44,1997,Szwecja,51.733,55.760
3195,44,1998,Szwecja,52.883,51.733
3196,44,1999,Szwecja,47.188,52.883


In [4]:
current_model = "basic"
variant_co2 = 'co2'

global_csv_path = os.path.join(config.predictions, f'combined_results_{variant_co2}.csv')
global_results = GlobalResults(global_csv_path, keys=["country", "year", "set"])

train = train_df.copy()
test = test_df.copy()

if "year" not in train.columns:
    train = train.reset_index()
if "year" not in test.columns:
    test = test.reset_index()

train["set"] = "train"
test["set"] = "test"

train = train.rename(columns={
    "co2_predicted": f"co2_predicted_{current_model}",
    "co2_actual": f"co2_actual_{current_model}"
})
test = test.rename(columns={
    "co2_predicted": f"co2_predicted_{current_model}",
    "co2_actual": f"co2_actual_{current_model}"
})

new_results_df = pd.concat([train, test], axis=0)
new_results_df = new_results_df.sort_values(by=["year", "country", "set"])

global_results.append_results(new_results_df)

## Metrics

In [5]:
evaluator = PredictionEvaluator()
evaluator.evaluate(train_df, test_df, actual_col='co2_actual', predicted_col='co2_predicted', variant = variant_co2, model_output_file=config.metrics_basic)